In [42]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import os
from PIL import Image
from tensorflow.keras.utils import img_to_array, load_img
import shutil

Image Directorys

In [43]:
input_dir = "G:/oil_spills_detection/Datasets"  # Path to the original dataset
output_dir = "G:/oil_spills_detection/Datagen"  # Directory to save augmented images
validation_dir = "G:/oil_spills_detection/Validation_data"
train_dir= "G:/oil_spills_detection/train_data"

Splitting data into train and test

In [17]:

for class_dir in os.listdir(input_dir):
    class_path = os.path.join(input_dir, class_dir)
    
    train_class_dir = os.path.join(train_dir, class_dir)
    val_class_dir = os.path.join(validation_dir, class_dir)
    output_class_dir =os.path.join(output_dir, class_dir)
    

    images = [f for f in os.listdir(class_path) if os.path.isfile(os.path.join(class_path, f)) and f.endswith(('.jpg', '.png', '.jpeg'))]
    
    train_images, val_images = train_test_split(images, test_size = 0.2 , shuffel= True, random_state = 42)
    for img_file in train_images:
        src = os.path.join(class_path, img_file)
        dst = os.path.join(train_class_dir, img_file)
        opt = os.path.join(output_class_dir, img_file)
        shutil.copy(src,dst)
        shutil.copy(src, opt)

    for img_file in val_images:
         src= os.path.join(class_path, img_file)
         dst = os.path.join(val_class_dir, img_file) 
         shutil.copy(src,dst)   
    



Data Argumentation

In [53]:
train_data_gen = ImageDataGenerator(
    rotation_range=30,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip= True,
    fill_mode='constant'
)

Apply argumentation on data

In [54]:
for class_dir in os.listdir(train_dir):
    class_path = os.path.join(train_dir, class_dir)
    if not os.path.isdir(class_path):
        continue
    augmented_class_dir = os.path.join(output_dir, class_dir)
    os.makedirs(augmented_class_dir, exist_ok=True)
    # Get list of training images
    images = [f for f in os.listdir(class_path) if os.path.isfile(os.path.join(class_path, f)) and f.endswith(('.jpg'))]
    num_images = len(images)
    arg_image = [[f for f in os.listdir(augmented_class_dir) if os.path.isfile(os.path.join(augmented_class_dir, f)) and f.endswith(('.jpg'))]]
    num_arg_images = len(arg_image)

    # Desired number of images per class (adjust as needed)
    desired_count = 2980

    # Augment images if needed
    if num_arg_images < desired_count:
        generated_count=0
        for img_file in images:
            img_path = os.path.join(class_path, img_file)
            img = load_img(img_path)
            x = img_to_array(img)
            x = np.expand_dims(x, axis=0)

            # Generate augmented images
            for  batch in train_data_gen.flow(x, batch_size=1, save_to_dir=augmented_class_dir, save_prefix='aug', save_format='jpg'):
                generated_count +=1
                if num_images + generated_count >= desired_count:
                    break
            if num_images + generated_count >= desired_count:
                break  

print("Data augmentation completed.")

Data augmentation completed.
